In [1]:
# Import libs

import numpy as np
import nrrd
import matplotlib.pyplot as plt
import csv

In [1]:
# Input values - these are files saved by 3D Slicer

path_to_label_map = "Initsegmentation-label.nrrd"
path_to_data_matrix = "3 PET WB.nrrd"
path_to_colortable = "Initsegmentation-label_ColorTable.ctbl"

path_to_output_volume = "40Percent_threshold.nrrd"
path_to_output_map = "40Percent_threshold_labelmap.nrrd"

In [ ]:
labelmap, header = nrrd.read(path_to_label_map)
data_matrix, data_matrix_header = nrrd.read(path_to_data_matrix)

# Get the labels of the structures from the color-table file 
# This is not strictly needed but nice for easier handling

f = open(path_to_colortable, "r")

list_of_names = []
list_of_labels = []

for line in f:
    if line[0] != "#":
        row_vector = (line.split(" "))
        name = row_vector[1]
        label = row_vector[0]
        
        print(name, label)
        
        list_of_names.append(name)
        list_of_labels.append(label)
        
zip_iterator = zip(list_of_labels, list_of_names)

labelNames = dict(zip_iterator)

# Initialize two volumes to write the output

output_matrix = np.zeros(np.shape(data_matrix))
output_labelmap = np.zeros(np.shape(data_matrix))

assert np.shape(labelmap) == np.shape(data_matrix) and np.shape(labelmap) == np.shape(output_matrix)

# Ensure that spatial and origin are ok
data_header_output = data_matrix_header
data_header_output['type'] = 'float'

labels_in_map = np.arange(np.max(labelmap))+1

# Threshold structure by structure
# To be a part of the output the voxel must both be above a certain threshold _and_ be in that segment

# Set a figure to plot results when applicable

fig = plt.figure()
ax = fig.add_subplot(111)

voxel_list = [] # list to hold voxel-values if one wants to see the distribution
tick_list = [] # List to hold the labels of the tick

for label in labels_in_map:
    
    I = labelmap == label # Index to the labels
    
    #==================================================================================
    #
    # Insert your own analysis here - example with a simple thresholding is shown
    #
    #==================================================================================
    lesion_threshold = 0.4*np.max(data_matrix[I]) # Find the threshold value
    above_threshold = data_matrix>lesion_threshold # Index of voxels above the threshold
    threshold_mask = np.logical_and(I, above_threshold) # The overlap of indexes of threshold and label
    output_matrix[threshold_mask] = data_matrix[threshold_mask] # Set the values to an output volume
    output_labelmap[threshold_mask] = label 
    voxel_list.append(data_matrix[threshold_mask])
    
    tick_list.append(labelNames[str(label)])

plt.boxplot(voxel_list)
ax.set_xticklabels(tick_list)

# Save the volumes for further inspection
nrrd.write(path_to_output_volume, output_matrix, data_header_output, index_order='F')
nrrd.write(path_to_output_map, output_labelmap, header, index_order='F')